In [1]:
# ! pip install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.30.2 trl==0.4.7 -q

In [1]:
import torch
print(torch.__version__)

2.0.0+cu118


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# THIS CELL SHOULD NOT GIVE ANY ERROR
!python -c "import bitsandbytes as _; print(_.__path__)"

['/opt/conda/lib/python3.10/site-packages/bitsandbytes']


In [4]:
# /opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup$
# https://github.com/TimDettmers/bitsandbytes/issues/620

# adding GOOGLE_VM_CONFIG_LOCK_FILE

# to the set ignorable inside def to_be_ignored in this 
# file lib/python3.10/site-packages/bitsandbytes/cuda_setup/env_vars.py

In [5]:
import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

data = load_dataset("timdettmers/openassistant-guanaco", split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [6]:
torch.cuda.is_available()

True

In [7]:
import os

os.environ['PJRT_DEVICE'] = 'GPU'

In [11]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer.pad_token = tokenizer.eos_token

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
)

In [16]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=bnb_config
)

model.config.use_cache=False
model.config.pretraining_tp=1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
device = torch.device("cuda")
model.to(device)

ValueError: `.to` is not supported for `4-bit` or `8-bit` models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [9]:
peft_config = LoraConfig(
    r=64, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)

In [10]:
training_arguments = TrainingArguments(
    output_dir="llama2_finetuned_chatbot",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    lr_scheduler_type="linear",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=1,
    max_steps=10,
    fp16=True,
    push_to_hub=False
)

In [11]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=512
)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Exception in thread Thread-8 (_worker):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch_xla/distributed/parallel_loader.py", line 166, in _worker
    batch = xm.send_cpu_data_to_device(batch, device)
  File "/opt/conda/lib/python3.10/site-packages/torch_xla/core/xla_model.py", line 1092, in send_cpu_data_to_device
    return ToXlaTensorArena(convert_fn, select_fn).transform(data)
  File "/opt/conda/lib/python3.10/site-packages/torch_xla/core/xla_model.py", line 431, in transform
    self._convert()
  File "/opt/conda/li

In [ ]:
trainer.push_to_hub()